In [10]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser


In [16]:
llm = Ollama(
    model="agent_pm", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

llm_json = Ollama(
    model="agent_json_fixer", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

In [14]:
timeline = ResponseSchema(
        name="timeline",
        description="The Project timeline with scope, as a bulletpoint list.",
    )
deliverables =  ResponseSchema(
        name="deliverables",
        description="The Project deliverables with scope, as a bulletpoint list.",
    )
team = ResponseSchema(
        name="team",
        description="The required Project team's skills and experience level, as a bulletpoint list.",
    )
risks = ResponseSchema(
        name="risks",
        description="The Project risks, as a bulletpoint list.",
    )
budget = ResponseSchema(
        name="budget",
        description="The Project budget for each task, as a bulletpoint list.",
    )
metrics = ResponseSchema(
        name="metrics",
        description="The Project metrics, as a bulletpoint list.",
    )
output_parser = StructuredOutputParser.from_response_schemas([
    timeline,
    deliverables,
    team,
    risks,
    budget,
    metrics,
])

response_format = output_parser.get_format_instructions()
print(response_format)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"timeline": string  // The Project timeline with scope, as a bulletpoint list.
	"deliverables": string  // The Project deliverables with scope, as a bulletpoint list.
	"team": string  // The required Project team's skills and experience level, as a bulletpoint list.
	"risks": string  // The Project risks, as a bulletpoint list.
	"budget": string  // The Project budget for each task, as a bulletpoint list.
	"metrics": string  // The Project metrics, as a bulletpoint list.
}
```


In [15]:
template = """\
You have been given the followin project brief. Plan the project tasks step by step providing timelines and resource skills required to deliver on each task.

{format_instructions}

Project Brief:
{brief}
"""

brief = """\
- The project is for a big multinational firm.
- The firm employs about 10000 people.
- The firm is grouped into a hierarchical structure, where each group specialises in a different industry or competency.
- The firm is struggling to forecast its revenue accurately.
- We are proposing to use machine learning an algorithms coupled with sourcing external market data to improve forecasting.
- We aim to achieve a better accuracy than their current forecasting process.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["brief"],
    partial_variables={"format_instructions": response_format}
)

_input = prompt.format_prompt(brief=brief)
output = llm(_input.to_string())
output_parser.parse(output)

```json
{
  "timeline": "```
	1. Define project scope and objectives (2 weeks)
	2. Identify relevant market data sources (2 weeks)
	3. Collect and preprocess historical data (4 weeks)
	4. Develop machine learning model (8 weeks)
	5. Train and validate model (6 weeks)
	6. Implement model in production environment (4 weeks)
	7. Monitor and evaluate model performance (4 weeks)
	8. Report final results to stakeholders (2 weeks)
```
"deliverables": "```
	1. Project scope and objectives document
	2. List of relevant market data sources
	3. Collected and preprocessed historical data
	4. Machine learning model
	5. Model training and validation reports
	6. Model implementation plan
	7. Performance monitoring and evaluation report
	8. Final project report
```
"team": "```
	1. Project manager with experience in managing similar projects
	2. Data scientist with expertise in machine learning and data analysis
	3. Data engineer with experience in collecting and preprocessing large datasets
	4. Softw

OutputParserException: Got invalid JSON object. Error: Invalid control character at: line 2 column 19 (char 20)

In [18]:
json_template = """\
You have been given the following text. Convert it to correct JSON. No other text or information should be provided.

{format_instructions}

Text to convert to JSON:
{text_not_yet_json}
"""

prompt = PromptTemplate(
    template=json_template,
    input_variables=["text_not_yet_json"],
    partial_variables={"format_instructions": response_format}
)

_input = prompt.format_prompt(text_not_yet_json=output)
json_output = llm_json(_input.to_string())
output_parser.parse(json_output)

Here is the converted JSON text:
```json
{
  "timeline": "1. Define project scope and objectives (2 weeks) \n2. Identify relevant market data sources (2 weeks) \n3. Collect and preprocess historical data (4 weeks) \n4. Develop machine learning model (8 weeks) \n5. Train and validate model (6 weeks) \n6. Implement model in production environment (4 weeks) \n7. Monitor and evaluate model performance (4 weeks) \n8. Report final results to stakeholders (2 weeks)",
  "deliverables": "1. Project scope and objectives document \n2. List of relevant market data sources \n3. Collected and preprocessed historical data \n4. Machine learning model \n5. Model training and validation reports \n6. Model implementation plan \n7. Performance monitoring and evaluation report \n8. Final project report",
  "team": "1. Project manager with experience in managing similar projects \n2. Data scientist with expertise in machine learning and data analysis \n3. Data engineer with experience in collecting and prep

{'timeline': '1. Define project scope and objectives (2 weeks) \n2. Identify relevant market data sources (2 weeks) \n3. Collect and preprocess historical data (4 weeks) \n4. Develop machine learning model (8 weeks) \n5. Train and validate model (6 weeks) \n6. Implement model in production environment (4 weeks) \n7. Monitor and evaluate model performance (4 weeks) \n8. Report final results to stakeholders (2 weeks)',
 'deliverables': '1. Project scope and objectives document \n2. List of relevant market data sources \n3. Collected and preprocessed historical data \n4. Machine learning model \n5. Model training and validation reports \n6. Model implementation plan \n7. Performance monitoring and evaluation report \n8. Final project report',
 'team': '1. Project manager with experience in managing similar projects \n2. Data scientist with expertise in machine learning and data analysis \n3. Data engineer with experience in collecting and preprocessing large datasets \n4. Software enginee